In [8]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

In [45]:
# X = pd.read_csv('X.csv')
# test_X = pd.read_csv('test_X_dash.csv')
# y = X['Purchase']
# X.drop('Purchase', axis =1, inplace = True)

X = X_dash
X = X.as_matrix()
y = y.as_matrix()
test_X = test_X_dash.as_matrix()

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

# scale = StandardScaler()
# X_train = scale.fit_transform(X_train)
# X_test = scale.fit_transform(X_test)

In [5]:
# define base model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(13, input_dim=9, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# 4344.69865067 RMSE for y_test

In [63]:
# define the model
def larger_model():
    # create model
    model = Sequential()
    model.add(Dense(9, input_dim=9, kernel_initializer='normal', activation='relu'))
    model.add(Dense(18, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model
# with 13
# 4201.84651119 with mean as fillna batch_size =5, nb_epoch = 100
# with 20
# 4624.18753801 with 0 as fillna, batch_size = 5, nb_epoch = 100
# with 9,18,18
# 4189.97370874 with 0 as fillna, batch_size = 5, nb_epoch = 50
# 4166.32064455 with 0 as fillna, batch_size = 5, nb_epoch = 60
# 4206.06987428 with 0 as fillna, batch_size = 5, nb_epoch = 100
# with 9,18

In [51]:
# define wider model
def wider_model():
    # create model
    model = Sequential()
    model.add(Dense(20, input_dim=9, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model
# with 20 
# 4627.50892767 with 0 as fillna and batch_size = 10 , nb_epoch = 100
# 4227.00458626 with 0 as fillna and batch_size = 5 , nb_epoch = 100

In [64]:
estimator = KerasRegressor(build_fn= larger_model, nb_epoch=100, batch_size=5,verbose=0)

estimator.fit(X_train, y_train)
y_pred = estimator.predict(X_test)

print(np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

4629.59426037


In [53]:
y_pred

array([ 13553.98730469,   9282.99804688,   7119.62451172, ...,
        13243.70507812,   7791.90869141,   6988.27587891], dtype=float32)

In [54]:
test = pd.read_csv('test.csv')
test_output = test[['User_ID', 'Product_ID']]
test_output['Purchase'] = y_pred
test_output.to_csv('keras-wider.csv', index = False)

/home/greed/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


## Doing preprocessing such that filling with zero instead of -999

In [12]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [41]:
df = pd.read_csv('train.csv')

In [42]:
# df['Product_Category_2'].fillna(df['Product_Category_2'].mean(), inplace = True)
# df['Product_Category_3'].fillna(df['Product_Category_3'].mean(), inplace = True)
df['Product_Category_2'].fillna(0, inplace = True)
df['Product_Category_3'].fillna(0, inplace = True)

In [43]:
X = df.drop(['Purchase','User_ID', 'Product_ID'], axis = 1)
X_t = X.drop(['Gender', 'Age','Occupation','City_Category','Stay_In_Current_City_Years'], axis = 1)
X2 = X.drop(['Marital_Status','Product_Category_1','Product_Category_2', 'Product_Category_3'], axis = 1)
X_dash = X2.apply(LabelEncoder().fit_transform).join(X_t)

In [44]:
X_dash.head()

,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
0,0,0,10,0,2,0,3,0.0,0.0
1,0,0,10,0,2,0,1,6.0,14.0
2,0,0,10,0,2,0,12,0.0,0.0
3,0,0,10,0,2,0,12,14.0,0.0
4,1,6,16,2,4,0,8,0.0,0.0


In [32]:
y = df.Purchase

In [34]:
test = pd.read_csv('test.csv')

In [35]:
test_X = test.drop(['User_ID', 'Product_ID'], axis = 1)
test_X_t = test_X.drop(['Gender', 'Age','Occupation','City_Category','Stay_In_Current_City_Years'], axis = 1)
test_X2 = test_X.drop(['Marital_Status','Product_Category_1','Product_Category_2', 'Product_Category_3'], axis = 1)
test_X_dash = test_X2.apply(LabelEncoder().fit_transform).join(test_X_t)

In [36]:
test_X_dash['Product_Category_2'].fillna(0, inplace = True)
test_X_dash['Product_Category_3'].fillna(0, inplace = True)

In [37]:
test_X_dash

,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
0,1,4,7,1,2,1,1,11.0,0.0
1,1,2,17,2,0,0,3,5.0,0.0
2,0,3,1,1,4,1,5,14.0,0.0
3,0,3,1,1,4,1,4,9.0,0.0
4,0,2,1,2,1,0,4,5.0,12.0
5,1,4,1,2,3,1,2,3.0,15.0
6,1,4,1,2,3,1,1,11.0,15.0
7,1,4,1,2,3,1,2,4.0,9.0
8,1,2,7,0,1,0,10,13.0,16.0
9,1,1,15,0,4,0,5,14.0,0.0


In [38]:
X.shape, y.shape, test_X.shape

((550068, 9), (550068,), (233599, 9))

In [39]:
y.as_matrix()

array([ 8370, 15200,  1422, ...,   137,   365,   490])